## Langchain with neo4j

validate the Aura Instance is available
NEO4J_URI=neo4j+s://838f5909.databases.neo4j.io
NEO4J_USERNAME=neo4j
NEO4J_PASSWORD=jTem1jaUGnBlvHqBz8hKZMg9CKXvEmH55Sj3XT5nyRg
NEO4J_DATABASE=neo4j
AURA_INSTANCEID=838f5909
AURA_INSTANCENAME=Instance01

In [1]:
NEO4J_URI="neo4j+s://838f5909.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="jTem1jaUGnBlvHqBz8hKZMg9CKXvEmH55Sj3XT5nyRg"

In [2]:
import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [3]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

/var/folders/9n/z6chq_9d49n0jsk9yn1f31lr0000gn/T/ipykernel_25273/2377705909.py:3: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the `langchain-neo4j package and should be used instead. To use it run `pip install -U `langchain-neo4j` and import as `from `langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)


In [4]:
movie_data_url = "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv"

In [5]:
movie_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row
MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
m.title = row.title,
m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|')|
    MERGE(p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, "|")|
    MERGE(p:Person {name:trim(actor)})
    MERGE(p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres,'|')|
    MERGE(g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [6]:
graph.query(movie_query)

[]

In [7]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [8]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")


In [23]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key=groq_api_key,model_name = "openai/gpt-oss-120b")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x11edb3e30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x11f00d8e0>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [24]:
from langchain_classic.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True,allow_dangerous_requests=True)

chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x104c0f7d0>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x11edb3e30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x11f00d8e0>, model_name='openai/gpt-oss-120b'

In [33]:
response = chain.invoke({"query":"Who is appeared more time as a director and actor?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[r]->(:Movie)
WHERE type(r) IN ['DIRECTED','ACTED_IN']
WITH p, count(r) AS total
RETURN p.name AS name, total
ORDER BY total DESC
LIMIT 1
Full Context:
[{'name': 'Kenneth Branagh', 'total': 5}]

> Finished chain.


In [34]:
response

{'query': 'Who is appeared more time as a director and actor?',
 'result': 'Kenneth\u202fBranagh appears the most as a director and actor.'}